In [ ]:
!pip install ray===2.7.1

In [ ]:
import ray
import tensorflow as tf
import os
import math
import matplotlib.pyplot as plt

In [ ]:
os.cpu_count()

2

In [ ]:
ray.init(num_cpus=2, num_gpus=0)

2023-11-06 03:05:51,917	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.7.1


# Data processing

For the homework, you should download the dataset with the provided link. The provided dataset has already partitioned the data by clients.

In [ ]:
NUM_CLIENT = 100

def partition_data():
    # Preprocessing
    ...

    return partitions, testset


# Define model

In [ ]:
def get_model():
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )
    model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Client class

In [ ]:
@ray.remote(num_cpus=1, num_gpus=0)
class Client:
    def __init__(self, local_data):
        # Create model
        self.model = get_model()

        # Set data
        x_local, y_local = local_data

        # TODO: Split 80% of the client's data for training, 20% for validation
        ...

        return

    def train(self, global_weights):
        # TODO: Perform E epochs of training using the local traininig data
        ...



    def evaluate(self):
        # TODO: Use the updated model,
        #       compute the loss and accuracy on the local train and validation data
        ...


In [ ]:
# Evaluation on held-out test data

def test(test_data, weights):
    model = get_model()  # Construct the model
    model.set_weights(weights)  # Set model weights with the latest parameters

    # TODO: Get test accuracy
    ...

    return test_accuracy

# Aggregated metrics and losses
Aggregate client metrics and losses for (federated) evaluation metrics.

In [ ]:
def weighted_average(client_results):
    # TODO: Aggregate client losses or accuracies by taking a weighted average.
    #       Weights are proportional to the number of samples at each client
    ...

    return aggregated_value

# Aggregation of client updates

In [ ]:
def fedAvg(client_weights):
    # TODO: Implement FedAvg
    ...

    return avg_weights

# Launch the learning

In [ ]:
# Initialize the global model weights
global_model = get_model()

# Get data
data_by_clients, test_data = partition_data()

total_rounds = 100

C = 0.1   # Only a fraction of C of clients participate in the local training per round
E = 10     # Number of epochs for each local training

for round in range(total_rounds):
    # Step 1
    # TODO: Sample C fraction of clients
    # TODO: Get the local data for the selected clients
    sampled_clients_data = ...

    # Create Client instances for selected clients and assign local data
    clients = [Client.remote(data) for data in sampled_clients_data]

    # Step 2: Perform local training and evaluaton on the clients
    client_weights = ray.get([client.train.remote(global_model.get_weights()) for client in clients])
    client_results = ray.get([client.evaluate.remote() for client in clients])

    # Step 3
    # TODO: Aggregate the losses and metrics, keep track of the metrics
    weighted_average(client_results)

    # Step 4
    # TODO: Aggregate the client updates
    weighted_avg_weights = fedAvg(client_weights)

    # Step 5
    # Update the global model
    global_model.set_weights(weighted_avg_weights)


# Evaluate on test data after training
test(test_data, global_model.get_weights())

# Create plots